The same as "pytorch_training_loop" but better :)

In [2]:
import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from model import *
from data import *
from fastai.text.all import *

In [3]:
df_x = pd.read_csv('data/toy_x.csv')
df_y = pd.read_csv('data/toy_y.csv')
df_x.shape, df_y.shape

((19010, 4097), (19010, 2))

In [4]:
class FastDs(Dataset):
    def __init__(self, df_x, df_y):
        self.x = df_x
        self.y = df_y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        x = self.x.iloc[idx][1:]
        y = self.y.iloc[idx][1:]
        return torch.Tensor(x).unsqueeze(0), torch.Tensor(y).to(torch.long)

In [5]:
train_ds = FastDs(df_x[:-2000], df_y[:-2000])
valid_ds = FastDs(df_x[-2000:], df_y[-2000:])
len(train_ds), len(valid_ds)

(17010, 2000)

In [6]:
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, batch_size=16, shuffle=False, drop_last=True)
len(train_dl), len(valid_dl)

(1063, 125)

In [7]:
#DataLoade

In [7]:
batch = next(iter(train_dl))
len(batch), batch[0].shape, batch[1].shape

(2, torch.Size([16, 1, 4096]), torch.Size([16, 1]))

In [10]:
model = EffNet_b0(out_feats=16)
out = model(batch[0])
out.shape

torch.Size([16, 18, 16])

In [12]:
#model

In [10]:
dls = DataLoaders(train_dl, valid_dl)

As a reminder, we're just playing around here -- a proper validation set should be used and not just the same dataloader as the training set :)

In [14]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss())

Trying to train this model will not work -- out of the box -- this can be confirmed by uncommenting the line below

In [9]:
#learn.fit(1)

The callback below isn't needed since we can manipulate our dataset here

In [19]:
class compress_cb(Callback):
    def after_pred(self):
        self.learn.pred = self.pred.squeeze(2)

In [20]:
dls.to('cuda');
model.to('cuda');

In [36]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features)).to('cuda')
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        print(input.shape)
        print(label.shape)
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        #sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        sine = torch.sqrt(1.0 - torch.pow(cosine,2)).to(cosine.dtype)
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        print(one_hot.shape)
        #one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [29]:
F.normalize(out.squeeze(2)).shape

torch.Size([16, 18, 16])

In [35]:
F.normalize(batch[1].to(torch.float)).shape

torch.Size([16, 1])

In [22]:
F.linear(out.squeeze(2), batch[1].squeeze(1).to(torch.float)).shape

torch.Size([16, 18])

In [18]:
out.squeeze(2).shape, batch[1].squeeze(1).shape

(torch.Size([16, 18, 16]), torch.Size([16]))

In [27]:
F.linear(F.normalize(out.squeeze(2)), batch[1].squeeze(1).to(torch.float)).shape

torch.Size([16, 18])

In [38]:
ArcMarginProduct(16, 512)(out.squeeze(2).to('cuda'), batch[1].squeeze(1).to(torch.float).to('cuda')).shape

torch.Size([16, 18, 16])
torch.Size([16])
torch.Size([16, 18, 512])


torch.Size([16, 18, 512])

In [21]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss(),
               cbs=[compress_cb], 
                metrics=[accuracy])#.to_fp16()

In [16]:
#for x in range(len(valid_dl)):
#    batch = next(iter(valid_dl))
#    out = learn.model(batch[0])
#    print(nn.CrossEntropyLoss()(out, batch[1]))

In [ ]:
#out.dtype

In [40]:
#learn.lr_find()

In [39]:
#learn.fine_tune(1)